In [52]:
module Map =
    let mapValueAtKey f key map =
        map
        |> Map.map (
            fun k v ->
                if key = k then
                    f v
                else 
                    v
            )

module Stacks =
    let init keys =
        keys
        |> Seq.map (fun k -> k, [])
        |> Map.ofSeq
    
    let pushAt key value stacks =
        stacks
        |> Map.mapValueAtKey (fun s -> value::s) key

    let popAt key stacks =
        let popped = 
            stacks 
            |> Map.find key 
            |> List.head
        
        let rest =
            stacks
            |> Map.mapValueAtKey List.tail key
        
        (popped, rest)

In [77]:
#r "nuget: FSharp.Text.RegexProvider"

open System
open FSharp.Text

type Move = { Count : int; From : int; To : int }
type MoveRegex = FSharp.Text.RegexProvider.Regex< @"^move (?<Count>\d+) from (?<From>\d+) to (?<To>\d+)$" >
type BoxRegex = FSharp.Text.RegexProvider.Regex< @"^\[(?<Contents>[A-Z])\]?" >
let moveRegex = MoveRegex()
let boxRegex = BoxRegex()

let parseText (lines : string[]) =
    let header =
        lines 
        |> Seq.takeWhile (not << String.IsNullOrEmpty)
        |> List.ofSeq
        |> List.rev

    let moves = 
        lines
        |> Array.skip (header.Length + 1)
        |> Array.map (
            fun l -> 
                let m = moveRegex.TypedMatch l
                { Count = int m.Count.Value; From = int m.From.Value; To = int m.To.Value }
            )

    let stackNames = 
        let labels = List.head header
        labels.Split([|' '|], StringSplitOptions.RemoveEmptyEntries)
        |> Array.map int

    let stacks =
        stackNames
        |> Stacks.init

    let boxes = 
        header
        |> List.skip 1
        |> Seq.collect (
            fun row ->
                (row :> char seq)
                |> Seq.chunkBySize 4 
                |> Seq.map String
                |> Seq.zip stackNames
                |> Seq.choose (fun (n, t) -> boxRegex.TryTypedMatch t |> Option.map (fun r -> n, r.Contents.Value[0]))
            )
    
    let stacks =
        boxes
        |> Seq.fold (
            fun s (k, v) ->
                Stacks.pushAt k v s
            ) stacks

    stacks, moves

let applyMove stacks (move : Move) =
    [1 .. move.Count]
    |> Seq.fold (
        fun s _ ->
            s
            |> Stacks.popAt move.From
            ||> Stacks.pushAt move.To
        ) stacks

let tops stacks =
    stacks 
    |> Map.map (fun _ s -> Seq.head s)
    |> Map.values
    |> Array.ofSeq
    |> String    


Installed Packages FSharp.Text.RegexProvider, 2.1.0

In [78]:
#r "nuget:FsUnit"

open FsUnitTyped

let lines = 
    [|
        "    [D]    "
        "[N] [C]    "
        "[Z] [M] [P]"
        " 1   2   3 "
        ""
        "move 1 from 2 to 1"
        "move 3 from 1 to 3"
        "move 2 from 2 to 1"
        "move 1 from 1 to 2"
    |]

let (stacks, moves) = parseText lines

moves |> shouldEqual [| { Count = 1; From = 2; To = 1 }; { Count = 3; From = 1; To = 3 }; { Count = 2; From = 2; To = 1}; { Count = 1; From = 1; To = 2} |]
stacks[1] |> List.ofSeq |> shouldEqual [ 'N'; 'Z' ]
stacks[2] |> List.ofSeq |> shouldEqual [ 'D'; 'C'; 'M' ]
stacks[3] |> List.ofSeq |> shouldEqual [ 'P' ]

let stacks1 = applyMove stacks moves[0]
stacks1[1] |> List.ofSeq |> shouldEqual [ 'D'; 'N'; 'Z' ]
stacks1[2] |> List.ofSeq |> shouldEqual [ 'C'; 'M' ]
stacks1[3] |> List.ofSeq |> shouldEqual [ 'P' ]

let stacks2 = applyMove stacks1 moves[1]
stacks2[1] |> List.ofSeq |> shouldEqual [ ]
stacks2[2] |> List.ofSeq |> shouldEqual [ 'C'; 'M' ]
stacks2[3] |> List.ofSeq |> shouldEqual [ 'Z'; 'N'; 'D'; 'P' ]


Installed Packages FsUnit, 5.1.0

In [79]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_05.txt")

let (stacks, moves) = 
    File.ReadAllLines(sourcePath) 
    |> parseText

let result = 
    moves
    |> Array.fold applyMove stacks
    |> tops

In [ ]:
printfn "Top boxes are %s" result

## Part 2

In [81]:
let applyMove9001 stacks (move : Move) =
    let steps = [1 .. move.Count]
    let (craneHolds, stacks) = 
        steps
        |> Seq.fold (
            fun (c,s) _ ->
                let (box, s') = Stacks.popAt move.From s
                (box::c, s')
            ) ([], stacks)

    craneHolds
    |> Seq.fold (
        fun s box ->
            Stacks.pushAt move.To box s
        ) stacks

let result = 
    moves
    |> Array.fold applyMove9001 stacks
    |> tops

In [ ]:
printfn "Top boxes are %s" result